# Phase 2: Intermediate - Active Trading & Advisory

In this phase, we evolve from simple sequential pipelines to **Parallel Agent Execution**. This allows us to gather data from multiple independent markets (Equities and Crypto) simultaneously before synthesizing a recommendation.

### System Architecture
We use a branching flow where data collection happens in parallel:
```
          ┌→ Market Data Provider →┐
START ----┤                        ├→ Recommender Agent → END
          └→ Binance Tracker ------→┘
```

| Agent | Role | Parallel Status |
|---|---|---|
| **MarketDataProvider** | Analyzes traditional stock equity (e.g., NVDA) | Yes |
| **BinanceTracker** | Analyzed crypto assets (e.g., BTC-USD) | Yes |
| **RecommenderAgent** | Synthesizes a multi-market strategy | No (Wait for both) |

## 1. Environment Setup

In [22]:
%pip install vinagent==0.0.6.post3

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv('.env'))

llm = ChatOpenAI(model="gpt-4o-mini")
print("LLM initialized.")

Note: you may need to restart the kernel to use updated packages.
LLM initialized.


## 2. Define Multi-Market State

The state now contains specific keys for each market stream.

In [23]:
from typing import Annotated, TypedDict

def append_messages(existing: list, update: dict) -> list:
    return existing + [update]

class TradingState(TypedDict):
    messages: Annotated[list[dict], append_messages]
    equity_insight: str # From MarketDataProvider
    crypto_insight: str # From BinanceTracker
    final_recommendation: str

In [24]:
from datetime import datetime
from vinagent.register import primary_function

@primary_function
def get_current_time() -> str:
    """
    Get the current date and time. Use this to know 'today's' date.
    Returns:
        str: Current date and time in YYYY-MM-DD HH:MM:SS format.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Custom time tool defined.")

Custom time tool defined.


## 3. Implement Parallel Agents

Notice that `MarketDataProvider` and `BinanceTracker` don't depend on each other's output.

In [25]:
from vinagent.multi_agent import AgentNode
from vinagent.logger.logger import logging_message

class MarketDataProvider(AgentNode):
    """Fetches and analyzes stock market data."""
    @logging_message
    def exec(self, state: TradingState) -> dict:
        # In a real scenario, this would use fetch_stock_data tool
        prompt = "Analyze NVDA's current market sentiment and technical outlook. Provide a summary for a trader."
        output = self.invoke(prompt)
        return {
            "messages": {"role": "MarketDataProvider", "content": output.content if hasattr(output, "content") else str(output)},
            "equity_insight": output.content if hasattr(output, "content") else str(output)
        }

class BinanceTracker(AgentNode):
    """Fetches and analyzes crypto market data."""
    @logging_message
    def exec(self, state: TradingState) -> dict:
        # Crypto symbols in yfinance use -USD suffix, e.g., BTC-USD
        prompt = "Analyze Bitcoin (BTC-USD) current performance and volatility. Provide a summary for a trader."
        output = self.invoke(prompt)
        return {
            "messages": {"role": "BinanceTracker", "content": output.content if hasattr(output, "content") else str(output)},
            "crypto_insight": output.content if hasattr(output, "content") else str(output)
        }

class RecommenderAgent(AgentNode):
    """Merges insights into a unified strategy."""
    @logging_message
    def exec(self, state: TradingState) -> dict:
        equity = state.get("equity_insight", "")
        crypto = state.get("crypto_insight", "")
        
        prompt = (
            f"Equity Analysis: \n{equity}\n\n"
            f"Crypto Analysis: \n{crypto}\n\n"
            f"Based on both markets, provide a unified multi-asset trading recommendation."
        )
        output = self.invoke(prompt)
        return {
            "messages": {"role": "Recommender", "content": output.content if hasattr(output, "content") else str(output)},
            "final_recommendation": output.content if hasattr(output, "content") else str(output)
        }

## 4. Assemble the Parallel Flow

We use the `START` constant to trigger multiple agents simultaneously.

In [26]:
from vinagent.multi_agent import CrewAgent
from vinagent.graph.operator import FlowStateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# 1. Define agents with standard tool registration (module paths)
instr = "CRITICAL: Format tool arguments as JSON object. NEVER use plain strings."

market_agent = MarketDataProvider(
    name="market_agent", 
    llm=llm, 
    tools=['vinagent.tools.yfinance_tools'], 
    instruction=instr
)

crypto_agent = BinanceTracker(
    name="crypto_agent", 
    llm=llm, 
    tools=['vinagent.tools.websearch_tools'], 
    instruction=instr
)

recommender = RecommenderAgent(
    name="recommender", 
    llm=llm, 
    tools=[], 
    instruction=instr
)

# 2. Register runtime tools manually (following Notebook 01 pattern)
for agent in [market_agent, crypto_agent, recommender]:
    agent.tools_manager.register_function_tool(get_current_time)

print("Agents instantiated with standardized tool registration.")

# 3. Build the crew
crew = CrewAgent(
    llm=llm,
    checkpoint=MemorySaver(),
    graph=FlowStateGraph(TradingState),
    flow=[
        START >> market_agent,
        START >> crypto_agent,
        market_agent >> recommender,
        crypto_agent >> recommender,
        recommender >> END
    ]
)
print("Phase 2 Crew Assembled with Direct Parallel Flow.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.register.tool:Registered fetch_stock_data:
{'tool_name': 'fetch_stock_data', 'arguments': {'symbol': 'str', 'start_date': '2020-01-01', 'end_date': '2025-01-01', 'interval': '1d'}, 'return': 'pd.DataFrame', 'docstring': "Fetch historical stock data from Yahoo Finance.\n\n    Args:\n        symbol (str): The stock symbol (e.g., 'AAPL' for Apple).\n        start_date (str): Start date for historical data (YYYY-MM-DD).\n        end_date (str): End date for historical data (YYYY-MM-DD).\n        interval (str): Data interval ('1d', '1wk', '1mo', etc.).\n\n    Returns:\n        pd.DataFrame: DataFrame containing historical stock prices.", 'dependencies': ['yfinance', 'pandas'], 'module_path': 'vinagent.tools.yfinance_tools', 'tool_type': 'module', 'tool_call_id': 'tool_2256c63b-bfce-4429-a39b-3124dedfaec'}
INFO:vinagent.register.tool:Registered visualize_stock_data:
{'tool_name': 'visual

Agents instantiated with standardized tool registration.
Phase 2 Crew Assembled with Direct Parallel Flow.


## 5. Execute Multi-Market Analysis

Run the system to see parallel processing in action.

In [27]:
from IPython.display import display, Markdown

query = "Compare NVDA and BTC performance for this week and suggest a 70/30 split strategy."

result = crew.invoke(
    query=query,
    user_id="analyst",
    thread_id=2
)

display(Markdown(f"""
### 📈 Multi-Asset Trading Recommendation
---
{result['final_recommendation']}
"""))

INFO:vinagent.multi_agent.crew:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:vinagent.agent.agent:Tool calling iteration 1/10


{'input': {'messages': {'role': 'user', 'content': 'Compare NVDA and BTC performance for this week and suggest a 70/30 split strategy.'}}, 'config': {'configurable': {'user_id': 'analyst'}, 'thread_id': 2}}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'fetch_stock_data', 'tool_type': 'module', 'arguments': {'symbol': 'BTC-USD', 'start_date': '2023-01-01', 'end_date': '2023-10-31', 'interval': '1d'}, 'module_path': 'vinagent.tools.yfinance_tools'}
INFO:vinagent.register.tool:Completed executing module tool fetch_stock_data({'symbol': 'BTC-USD', 'start_date': '2023-01-01', 'end_date': '2023-10-31', 'interval': '1d'})
INFO:vinagent.agent.agent:Tool calling iteration 2/10


Error fetching data for BTC-USD: Too Many Requests. Rate limited. Try after a while.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'search_api', 'tool_type': 'module', 'arguments': {'query': 'Bitcoin (BTC-USD) current performance and volatility analysis'}, 'module_path': 'vinagent.tools.websearch_tools'}
INFO:vinagent.register.tool:Completed executing module tool search_api({'query': 'Bitcoin (BTC-USD) current performance and volatility analysis'})
INFO:vinagent.agent.agent:Tool calling iteration 3/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 3 iterations.
INFO:vinagent.logger.logger:

============ BinanceTracker Response ============
{'messages': {'role': 'BinanceTracker', 'content': "Bitcoin's current price is $68,197.18, down 5% from last week. It has shown volatility, fluctuating between $62,400 and $74,500 in recent days. The market is active with

Error fetching data for AAPL: Too Many Requests. Rate limited. Try after a while.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'search_api', 'tool_type': 'module', 'arguments': {'query': 'multi-asset trading recommendations based on Bitcoin and Apple stock performance'}, 'module_path': 'vinagent.tools.websearch_tools'}
INFO:vinagent.register.tool:Completed executing module tool search_api({'query': 'multi-asset trading recommendations based on Bitcoin and Apple stock performance'})
INFO:vinagent.agent.agent:Tool calling iteration 3/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 3 iterations.
INFO:vinagent.logger.logger:

============ RecommenderAgent Response ============
{'messages': {'role': 'Recommender', 'content': "For your multi-asset trading strategy, considering Bitcoin's price volatility and the performance of Apple stock, a balanced approac


### 📈 Multi-Asset Trading Recommendation
---
For your multi-asset trading strategy, considering Bitcoin's price volatility and the performance of Apple stock, a balanced approach would be advisable. You might allocate approximately 1-2% of your portfolio to Bitcoin and 25% to Apple. It's also beneficial to implement equal-weight rebalancing strategies to manage risk and optimize returns effectively.
